<a href="https://colab.research.google.com/github/ZeyadSabbah/TrivagoRecommenderSystem/blob/master/FeatureSelection%26Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection & Modeling
## Mounting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem


## Loading Libraries & Datasets

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
from datetime import datetime
import re
import random
import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [0]:
TrainDataFilepath = '/content/drive/My Drive/Trivago/Datasets/clean_data/TrainData.csv'

TrainData = pd.read_csv(TrainDataFilepath)

## Using SelectKBest

TrainData has 20 features. Starting with just 15 important features to see the importance between the different features.

In [0]:
#declaring features and label
X_train = TrainData[['price', 'item_rank', 'price_rank', 'session_duration', 'item_duration', 'item_session_duration', 'item_interactions', 'maximum_step', 'top_list', 'NumberOfProperties',
       'NumberInImpressions', 'NumberInReferences', 'NumberAsClickout', 'NumberAsFinalClickout', 'FClickoutToImpressions', 'FClickoutToReferences', 'FClickoutToClickout', 'MeanPrice',
       'AveragePriceRank']]
y_train = TrainData[['clickout']]

In [7]:
bestfeatures = SelectKBest(score_func=chi2, k=15)
fit = bestfeatures.fit(X, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(15,'Score'))  #print 10 best features

                     Specs         Score
4            item_duration  8.882146e+08
6        item_interactions  2.450741e+07
11      NumberInReferences  1.650767e+07
12        NumberAsClickout  4.326980e+06
13   NumberAsFinalClickout  3.261157e+06
1                item_rank  2.560008e+06
5    item_session_duration  2.545062e+06
0                    price  8.438415e+05
8                 top_list  6.153390e+05
17               MeanPrice  2.273722e+05
10     NumberInImpressions  1.314937e+05
14  FClickoutToImpressions  1.062536e+05
9       NumberOfProperties  7.866433e+04
15   FClickoutToReferences  6.643049e+04
16     FClickoutToClickout  6.460986e+04


Scores are high which means the features are relevant to the output, so all of the features will be taken in modeling at first, and then by removing the least important features, the performance of the model will be captured.

TrainData is ready for processing and modeling, while validation and test sets still need to be engineered. There is a ready function that will transform sets into the same form of TrainData.
## Scaling Features

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])

In [7]:
from sklearn.compose import ColumnTransformer
full_pipeline = ColumnTransformer([
("num", num_pipeline, list(X_train))
])

X_train_scaled = full_pipeline.fit_transform(X_train)
X_train_scaled

array([[ 0.25239889, -1.60036787, -0.48211643, ...,  0.15662095,
         0.60099131,  0.24508405],
       [-0.54563347, -1.46058644, -1.46058644, ...,  0.03252758,
        -0.35485974, -0.54023804],
       [ 0.18249825, -1.32080501,  0.49635358, ...,  0.3204242 ,
        -0.10171608, -0.12754915],
       ...,
       [-0.18448014,  1.47482358, -0.20255357, ...,  0.05052112,
        -0.09796216,  0.27715247],
       [ 2.13389126,  1.61460501,  1.61460501, ...,  0.21246296,
         1.35274505,  0.53286415],
       [ 0.30482438,  1.75438644,  0.63613501, ...,  0.53634666,
         0.47849817,  0.35417802]])

## Models

###Without Resampling

In [0]:
def print_results(results):
  print('Best Prams: {}\n'.format(results.best_params_))

  means = results.cv_results_['mean_test_score']
  stds = results.cv_results_['std_test_score']
  for mean, std, params in zip(means, stds, results.cv_results_['params']):
    print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

#### Logistic Regression

In [12]:
t1  = datetime.now()
lr = LogisticRegression()
parameters = {
    'C':[0.01, 0.1, 1, 10]
}

cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(X_train_scaled, y_train.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'LR_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

Best Prams: {'C': 10}

0.965 (+/-0.0) for {'C': 0.01}
0.965 (+/-0.0) for {'C': 0.1}
0.965 (+/-0.0) for {'C': 1}
0.965 (+/-0.0) for {'C': 10}
Time taken :  0:25:42.234537


#### SVM

In [0]:
t1  = datetime.now()
svc = SVC()
parameters = {
    'kernel': ['linear', 'rbf'],
    'C':[0.1, 1, 10]
}
cv = GridSearchCV(svc, parameters, cv = 5)
cv.fit(X_train_scaled, y_train.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'SVC_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

#### Random Forest

In [0]:
t1 = datetime.now()
rf = RandomForestClassifier()
parameters = {
    'n_estimators':[5, 50, 250],
    'max_depth':[2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(X_train_scaled, y_train.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'RF_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

#### XGBoost

In [0]:
t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(X_train_scaled, y_train.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'XGB_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

### With SMOTE

### With Undersampling

In [0]:
ClickoutLen = len(y_train[y_train.clickout==1])
print(ClickoutLen)

In [0]:
NotClickoutIndices = y_train[y_train.clickout==0].index


In [0]:
ClickoutIndices = y_train[y_train.clickout==1].index

In [0]:
NotClickoutIndices = y_train[y_train.clickout==0].index
ClickoutIndices = y_train[y_train.clickout==1].index

NotClickoutRandomIndices = np.random.choice(NotClickoutIndices,
                                            ClickoutLen,
                                            replace=False)
UnderSampleIndices = np.concatenate([ClickoutIndecs, NotClickoutIndices])

XUnderSample = []
yUnderSample = []
for Index in UnderSampleIndices:
  XUnderSample.append(X_train_scaled[Index])
  yUnderSample.append(y_train[Index])

#### Logistic Regression

In [0]:
t1  = datetime.now()
lr = LogisticRegression()
parameters = {
    'C':[0.01, 0.1, 1, 10]
}

cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(XUnderSample, yUnderSample.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'LR_modelUndersampling.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

#### SVM

In [0]:
t1  = datetime.now()
svc = SVC()
parameters = {
    'kernel': ['linear', 'rbf'],
    'C':[0.1, 1, 10]
}
cv = GridSearchCV(svc, parameters, cv = 5)
cv.fit(XUnderSample, yUnderSample.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'SVC_modelUndersampling.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

#### Random Forest

In [0]:
t1 = datetime.now()
rf = RandomForestClassifier()
parameters = {
    'n_estimators':[5, 50, 250],
    'max_depth':[2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(XUnderSample, yUnderSample.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'RF_modelUnderSampling.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

#### XGBoost

In [0]:
t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(XUnderSample, yUnderSample.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'XGB_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))